In [1]:
import xarray as xr
import movingpandas as mpd
from datetime import timedelta

## Connect to the dataset

This just queries an overview of the dataset

In [2]:
ds = xr.open_dataset(
    "https://thredds.niva.no/thredds/dodsC/datasets/norsoop/color_fantasy/merged_acdd_color_fantasy.nc"
)
ds

<xarray.Dataset>
Dimensions:          (time: 1689725)
Coordinates:
  * time             (time) datetime64[ns] 2017-01-01T13:25:03 ... 2022-12-31...
    longitude        (time) float64 ...
    latitude         (time) float64 ...
Data variables:
    temperature      (time) float64 ...
    salinity         (time) float64 ...
    oxygen           (time) float64 ...
    temperature_qc   (time) float32 ...
    salinity_qc      (time) float32 ...
    oxygen_qc        (time) float32 ...
    trajectory_name  |S64 ...
Attributes: (12/42)
    title:                                     Ferrybox on MS Color Fantasy
    summary:                                   Ferry sailing from Oslo, Norwa...
    title_no:                                  FerryBox på MS Color Fantasy
    summary_no:                                Ferje fra Oslo, Norge til Kiel...
    date_created:                              2023-10-20T08:26:40Z
    keywords:                                  GCMDSK:EARTH SCIENCE > OCEANS ...
    ...                                        ...
    history:                                   Built with dscreator(https://g...
    source:                                    In Situ Ship-based station
    id:                                        14bb8759-81d8-4a1a-948a-14219d...
    _NCProperties:                             version=2,netcdf=4.7.4,hdf5=1....
    THREDDSMetadata_services_opendap_service:  https://thredds.niva.no/thredd...
    DODS_EXTRA.Unlimited_Dimension:            time

# Subset the data

If you don't need the full dataset it is important to subset the dataset to avoid downloading all the data. Select July data from 2022

In [3]:
ds_july = ds.sel(time="2022-07")
ds_july

<xarray.Dataset>
Dimensions:          (time: 32543)
Coordinates:
  * time             (time) datetime64[ns] 2022-07-01 ... 2022-07-31T23:59:00
    longitude        (time) float64 ...
    latitude         (time) float64 ...
Data variables:
    temperature      (time) float64 ...
    salinity         (time) float64 ...
    oxygen           (time) float64 ...
    temperature_qc   (time) float32 ...
    salinity_qc      (time) float32 ...
    oxygen_qc        (time) float32 ...
    trajectory_name  |S64 ...
Attributes: (12/42)
    title:                                     Ferrybox on MS Color Fantasy
    summary:                                   Ferry sailing from Oslo, Norwa...
    title_no:                                  FerryBox på MS Color Fantasy
    summary_no:                                Ferje fra Oslo, Norge til Kiel...
    date_created:                              2023-10-20T08:26:40Z
    keywords:                                  GCMDSK:EARTH SCIENCE > OCEANS ...
    ...                                        ...
    history:                                   Built with dscreator(https://g...
    source:                                    In Situ Ship-based station
    id:                                        14bb8759-81d8-4a1a-948a-14219d...
    _NCProperties:                             version=2,netcdf=4.7.4,hdf5=1....
    THREDDSMetadata_services_opendap_service:  https://thredds.niva.no/thredd...
    DODS_EXTRA.Unlimited_Dimension:            time

# Create a movingpandas trajectory

The dataset follows climate and forecast convention with featuretype trajectory, this makes it easy to create a trajectory in movingpandas also. 

In [4]:
traj = mpd.Trajectory(
    df=ds_july.to_dataframe(), traj_id=str(ds_july.trajectory_name.values), t="time", x="longitude", y="latitude"
)

In [5]:
traj.df.head()

,temperature,salinity,oxygen,temperature_qc,salinity_qc,oxygen_qc,trajectory_name,geometry,traj_id
time,,,,,,,,,
2022-07-01 00:00:00,18.96,21.177,NaN,1.0,1.0,9.0,b'color_fantasy',POINT (11.45366 57.51949),b'color_fantasy'
2022-07-01 00:01:00,18.95,21.184,NaN,1.0,1.0,9.0,b'color_fantasy',POINT (11.45100 57.52510),b'color_fantasy'
2022-07-01 00:02:00,18.96,21.184,NaN,1.0,1.0,9.0,b'color_fantasy',POINT (11.44834 57.53069),b'color_fantasy'
2022-07-01 00:03:00,18.97,21.190,NaN,1.0,1.0,9.0,b'color_fantasy',POINT (11.44580 57.53618),b'color_fantasy'
2022-07-01 00:04:00,19.00,21.198,NaN,1.0,1.0,9.0,b'color_fantasy',POINT (11.44321 57.54180),b'color_fantasy'


### Split the trajectory based on stops longer than 60min

In [6]:
split = mpd.ObservationGapSplitter(traj).split(gap=timedelta(minutes=60))
split

TrajectoryCollection with 32 trajectories

In [7]:
split.to_traj_gdf()

,traj_id,start_t,end_t,geometry,length,direction
0,b'color_fantasy'_0,2022-07-01 00:00:00,2022-07-01 07:38:00,"LINESTRING (11.45366 57.51949, 11.45100 57.525...",1.413322e+06,351.028602
1,b'color_fantasy'_1,2022-07-01 12:29:00,2022-07-02 07:01:00,"LINESTRING (10.64655 59.87608, 10.64019 59.872...",3.007600e+06,182.721134
2,b'color_fantasy'_2,2022-07-02 12:51:00,2022-07-03 07:37:00,"LINESTRING (10.21667 54.42160, 10.21983 54.425...",3.144820e+06,2.562277
3,b'color_fantasy'_3,2022-07-03 12:14:00,2022-07-04 07:01:00,"LINESTRING (10.64773 59.87718, 10.64071 59.873...",1.304175e+06,182.723659
4,b'color_fantasy'_4,2022-07-04 12:47:00,2022-07-05 07:37:00,"LINESTRING (10.21626 54.41832, 10.21938 54.421...",6.728270e+05,2.559019
5,b'color_fantasy'_5,2022-07-05 12:27:00,2022-07-06 07:01:00,"LINESTRING (10.64422 59.87505, 10.63814 59.871...",6.742561e+05,182.704932
6,b'color_fantasy'_6,2022-07-06 12:44:00,2022-07-07 07:37:00,"LINESTRING (10.21417 54.41597, 10.21747 54.419...",6.716665e+05,2.566254
7,b'color_fantasy'_7,2022-07-07 12:20:00,2022-07-08 07:00:00,"LINESTRING (10.64589 59.87593, 10.63818 59.872...",6.715646e+05,182.714187
8,b'color_fantasy'_8,2022-07-08 12:45:00,2022-07-09 07:40:00,"LINESTRING (10.21467 54.41676, 10.21772 54.419...",6.717968e+05,2.557910
9,b'color_fantasy'_9,2022-07-09 12:22:00,2022-07-10 07:02:00,"LINESTRING (10.64218 59.87401, 10.63482 59.870...",6.770324e+05,182.694369


# Plot the first segment of the trajectory 

In [9]:
split.trajectories[1].hvplot(
    c="salinity", title=ds.attrs["title"], line_width=2, frame_width=700, frame_height=500, colorbar=True, cmap="blues"
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]   (salinity)
   .Points.I :Points   [Longitude,Latitude]   (salinity,triangle_angle)